In [78]:
import numpy as np
import pandas as pd

In [79]:
data_set = pd.read_csv('../data/TrueFX/EUR-USD/datos_procesados_5T_allFeatures_EURUSD-2018-01.csv', parse_dates=True,
                      infer_datetime_format=True,index_col=0)

In [80]:
data_set.head()

,open,high,low,close,mean,std,var,median,sem,ask
2018-01-02 00:00:00,1.20094,1.20094,1.20011,1.20015,1.200530,0.000318,1.010923e-07,1.200435,0.000085,1.20110
2018-01-02 00:05:00,1.20015,1.20062,1.20011,1.20047,1.200343,0.000142,2.027103e-08,1.200300,0.000024,1.20084
2018-01-02 00:10:00,1.20039,1.20098,1.20035,1.20082,1.200752,0.000180,3.255425e-08,1.200775,0.000024,1.20126
2018-01-02 00:15:00,1.20080,1.20097,1.20025,1.20035,1.200719,0.000258,6.650895e-08,1.200860,0.000027,1.20091
2018-01-02 00:20:00,1.20034,1.20057,1.20031,1.20049,1.200501,0.000046,2.122442e-09,1.200510,0.000003,1.20084


In [81]:
data_set.columns = ['open','high','low','bid','Mean','Std','variance','Median','Sem','ask']

In [82]:
%run ../code/PNLEstimatorWrapper.py
%run ../code/EUtilities.py
EU = EUtilities()

In [83]:
window = 7
X, y_reg, bt = EU.build_dataset(data_set,window=window,bid_col='bid',binary_target=True,PNL=True)
X.shape

(6245, 9)

In [84]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00020
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00018


In [85]:
X['open'] = data_set.filter(X.index,axis=0).open
X['high'] = data_set.filter(X.index,axis=0).high
X['low'] = data_set.filter(X.index,axis=0).low
X['mean'] = data_set.filter(X.index,axis=0).Mean
#X['std'] = data_set.filter(X.index,axis=0).Std
X['variance'] = data_set.filter(X.index,axis=0).variance
X['median'] = data_set.filter(X.index,axis=0).Median
#X['sem'] = data_set.filter(X.index,axis=0).Sem

0.001535

In [86]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1,open,high,low,mean,variance,median
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00020,1.20048,1.20093,1.20045,1.200573,1.474732e-08,1.200550
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012,1.20079,1.20163,1.20073,1.201144,6.080010e-08,1.201115
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009,1.20129,1.20145,1.20129,1.201344,1.710837e-09,1.201330
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030,1.20143,1.20150,1.20134,1.201427,2.820158e-09,1.201430
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00018,1.20150,1.20150,1.20131,1.201361,3.642290e-09,1.201330


In [87]:
def compute_class_weigth(y):
    u = np.unique(y)
    r = {}
    for i in range(len(u)):
        r[i] = sum(u[0]==y)/sum(u[i]==y)
        
    return r

[         0,          1,          2,          3,          4,          5,
                6,    'PNL_0',    'PNL_1',     'open',     'high',      'low',
           'mean', 'variance',   'median']

In [88]:
X.columns

Index([         0,          1,          2,          3,          4,          5,
                6,    'PNL_0',    'PNL_1',     'open',     'high',      'low',
           'mean', 'variance',   'median'],
      dtype='object')

In [89]:
X.head()

,0,1,2,3,4,5,6,PNL_0,PNL_1,open,high,low,mean,variance,median
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,-0.00076,0.00020,1.20048,1.20093,1.20045,1.200573,1.474732e-08,1.200550
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,-0.00023,-0.00012,1.20079,1.20163,1.20073,1.201144,6.080010e-08,1.201115
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,-0.00020,-0.00009,1.20129,1.20145,1.20129,1.201344,1.710837e-09,1.201330
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,-0.00007,-0.00030,1.20143,1.20150,1.20134,1.201427,2.820158e-09,1.201430
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,-0.00028,-0.00018,1.20150,1.20150,1.20131,1.201361,3.642290e-09,1.201330


In [90]:
X = X.drop('std', axis=1)

ValueError: labels ['std'] not contained in axis

In [91]:
X = X.fillna(1)

In [92]:
from sklearn.naive_bayes import GaussianNB
    
y = bt
n_bdtrain = 4
n_bdtest = 1
mday = 1
wc0 = 0.06 #peso de la clase 0
wc1 = 0.06 #peso de la clase 1
wc2 = 0.44 #peso de la clase 2
wc3 = 0.44 #peso de la clase 3
#para indexar el dataframe por las fechas
dates = np.unique(X.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(priors=(wc0,wc1,wc2,wc3)), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)

df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2','%total'])
                      , index=dates)

df_GNB = df_GNB.dropna()

print('--------- GaussianNB 5T ----------------')
print('average PNL : ', np.mean(df_GNB.iloc[:,0]))
print('average buy: ', np.mean(df_GNB.iloc[:,1]))
print('average sell: ', np.mean(df_GNB.iloc[:,2]))
print('longest_buy: ', np.max(df_GNB.iloc[:,3]))
print('longest_sell: ', np.max(df_GNB.iloc[:,4]))
print('%correct_1: ', np.mean(df_GNB.iloc[:,5]))
print('%correct_0: ', np.mean(df_GNB.iloc[:,6]))
print('%correct_3: ', np.mean(df_GNB.iloc[:,7]))
print('%correct_2: ', np.mean(df_GNB.iloc[:,8]))
print('% correct total: ', np.mean(df_GNB.iloc[:,9]))
print('-------------------------------------')


wc0 = np.ones(df_GNB.shape[0])*wc0
wc1 = np.ones(df_GNB.shape[0])*wc1
wc2 = np.ones(df_GNB.shape[0])*wc2
wc3 = np.ones(df_GNB.shape[0])*wc3

df_GNB.insert(loc=10,column='wc0',value=wc0)
df_GNB.insert(loc=11,column='wc1',value=wc1)
df_GNB.insert(loc=12,column='wc2',value=wc2)
df_GNB.insert(loc=13,column='wc3',value=wc3)

print(df_GNB)

--------- GaussianNB 5T ----------------
average PNL :  -0.0015352631578950248
average buy:  164.73684210526315
average sell:  104.63157894736842
longest_buy:  285.0
longest_sell:  287.0
%correct_1:  0.3483028837912764
%correct_0:  0.05629453364169886
%correct_3:  0.24125879878336465
%correct_2:  0.37341909398696105
% correct total:  0.22019117911589886
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-08  0.00000  227.0   54.0           0.0          281.0    0.000000   
2018-01-09 -0.01253  281.0    0.0         281.0            0.0    1.000000   
2018-01-10 -0.00525  245.0   38.0         149.0           67.0    0.785047   
2018-01-11  0.00469  171.0  110.0          25.0          112.0    0.102362   
2018-01-12  0.00299  154.0  129.0         153.0          126.0    0.588652   
2018-01-15 -0.00505  280.0    0.0         280.0            0.0    1.000000   
2018-01-16  0.00000    0.0  283.0           0.0          283

/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:30: RuntimeWarning: invalid value encountered in long_scalars
  percent_1 = sum((pre==1)*(y==1))/sum(y==1) # si sube y gano
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:31: RuntimeWarning: invalid value encountered in long_scalars
  percent_0 = sum((pre==0)*(y==0))/sum(y==0) # si baja y gano
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:33: RuntimeWarning: invalid value encountered in long_scalars
  percent_2 = sum((pre==2)*(y==2))/sum(y==2) # si baja y pierdo
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:32: RuntimeWarning: invalid value encountered in long_scalars
  percent_3 = sum((pre==3)*(y==3))/sum(y==3) # si sube y pierdo


In [93]:
from sklearn.neighbors import KNeighborsClassifier

est_KNC = PNLEstimatorWrapper(KNeighborsClassifier(n_neighbors=7,n_jobs=2,leaf_size=30
                                                   ,algorithm='kd_tree',weights='distance'), PNL_column=['PNL_0','PNL_1'])
result_KNC = EU.step_validation(est_KNC, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_KNC, predict_KNC = EU.redim(result_KNC)
df_KNC = pd.DataFrame(result_KNC, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2','%total'])
                      , index=dates)

df_KNC = df_KNC.dropna()

print('--------- KNeighbors 5T ----------------')
print('average PNL : ', np.mean(df_KNC.iloc[:,0]))
print('average buy: ', np.mean(df_KNC.iloc[:,1]))
print('average sell: ', np.mean(df_KNC.iloc[:,2]))
print('longest_buy: ', np.max(df_KNC.iloc[:,3]))
print('longest_sell: ', np.max(df_KNC.iloc[:,4]))
print('%correct_1: ', np.mean(df_KNC.iloc[:,5]))
print('%correct_0: ', np.mean(df_KNC.iloc[:,6]))
print('%correct_3: ', np.mean(df_KNC.iloc[:,7]))
print('%correct_2: ', np.mean(df_KNC.iloc[:,8]))
print('% correct total: ', np.mean(df_KNC.iloc[:,9]))
print('-------------------------------------')

wc0 = 'distance'
wc1 = 'distance'
wc2 = 'distance'
wc3 = 'distance'

df_KNC.insert(loc=10,column='wc0',value=wc0)
df_KNC.insert(loc=11,column='wc1',value=wc1)
df_KNC.insert(loc=12,column='wc2',value=wc2)
df_KNC.insert(loc=13,column='wc3',value=wc3)

print(df_KNC)

/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:30: RuntimeWarning: invalid value encountered in long_scalars
  percent_1 = sum((pre==1)*(y==1))/sum(y==1) # si sube y gano
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:31: RuntimeWarning: invalid value encountered in long_scalars
  percent_0 = sum((pre==0)*(y==0))/sum(y==0) # si baja y gano
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:33: RuntimeWarning: invalid value encountered in long_scalars
  percent_2 = sum((pre==2)*(y==2))/sum(y==2) # si baja y pierdo
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:32: RuntimeWarning: invalid value encountered in long_scalars
  percent_3 = sum((pre==3)*(y==3))/sum(y==3) # si sube y pierdo


--------- KNeighbors 5T ----------------
average PNL :  -0.012937368421053044
average buy:  133.42105263157896
average sell:  135.94736842105263
longest_buy:  176.0
longest_sell:  176.0
%correct_1:  0.47726631539882686
%correct_0:  0.502675615727352
%correct_3:  0.014624505928853756
%correct_2:  0.03743809283975489
% correct total:  0.4196544129511034
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-08 -0.01619  228.0   53.0         176.0           13.0    0.792793   
2018-01-09 -0.01249  261.0   20.0         168.0            5.0    0.942149   
2018-01-10 -0.01196  114.0  169.0          21.0           47.0    0.364486   
2018-01-11 -0.01748   82.0  199.0           5.0           23.0    0.228346   
2018-01-12 -0.02276   28.0  255.0           3.0          176.0    0.085106   
2018-01-15 -0.00361  206.0   74.0          42.0            9.0    0.729508   
2018-01-16 -0.01611  143.0  140.0          18.0           14.0

In [29]:
from sklearn.tree import DecisionTreeClassifier


wc0 = 0.2 #peso de la clase 0
wc1 = 0.2 #peso de la clase 1
wc2 = 0.3 #peso de la clase 2
wc3 = 0.3 #peso de la clase 3
est_DTC = PNLEstimatorWrapper(DecisionTreeClassifier(class_weight={0:wc0,1:wc1,2:wc2,3:wc3})
                              , PNL_column=['PNL_0','PNL_1'])
result_DTC = EU.step_validation(est_DTC, X, y, EU.v_split(X,n_bdtrain,n_bdtest,mday))
result_DTC, predict_DTC = EU.redim(result_DTC)
df_DTC = pd.DataFrame(result_DTC, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2','%total'])
                      , index=dates)

df_DTC = df_DTC.dropna()

print('--------- GaussianNB 5T ----------------')
print('average PNL : ', np.mean(df_DTC.iloc[:,0]))
print('average buy: ', np.mean(df_DTC.iloc[:,1]))
print('average sell: ', np.mean(df_DTC.iloc[:,2]))
print('longest_buy: ', np.max(df_DTC.iloc[:,3]))
print('longest_sell: ', np.max(df_DTC.iloc[:,4]))
print('%correct_1: ', np.mean(df_DTC.iloc[:,5]))
print('%correct_0: ', np.mean(df_DTC.iloc[:,6]))
print('%correct_3: ', np.mean(df_DTC.iloc[:,7]))
print('%correct_2: ', np.mean(df_DTC.iloc[:,8]))
print('% correct total: ', np.mean(df_DTC.iloc[:,9]))
print('-------------------------------------')


wc0 = np.ones(df_DTC.shape[0])*wc0
wc1 = np.ones(df_DTC.shape[0])*wc1
wc2 = np.ones(df_DTC.shape[0])*wc2
wc3 = np.ones(df_DTC.shape[0])*wc3

df_DTC.insert(loc=10,column='wc0',value=wc0)
df_DTC.insert(loc=11,column='wc1',value=wc1)
df_DTC.insert(loc=12,column='wc2',value=wc2)
df_DTC.insert(loc=13,column='wc3',value=wc3)


print(df_DTC)

/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:30: RuntimeWarning: invalid value encountered in long_scalars
  percent_1 = sum((pre==1)*(y==1))/sum(y==1) # si sube y gano
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:31: RuntimeWarning: invalid value encountered in long_scalars
  percent_0 = sum((pre==0)*(y==0))/sum(y==0) # si baja y gano
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:33: RuntimeWarning: invalid value encountered in long_scalars
  percent_2 = sum((pre==2)*(y==2))/sum(y==2) # si baja y pierdo
/home/edwin/Documentos/Proyecto de grado/code/PNLEstimatorWrapper.py:32: RuntimeWarning: invalid value encountered in long_scalars
  percent_3 = sum((pre==3)*(y==3))/sum(y==3) # si sube y pierdo


--------- GaussianNB 5T ----------------
average PNL :  -0.006865263157895748
average buy:  151.0
average sell:  118.36842105263158
longest_buy:  52.0
longest_sell:  80.0
%correct_1:  0.45393470398524327
%correct_0:  0.37416377938600776
%correct_3:  0.17839156358295433
%correct_2:  0.07564612347079351
% correct total:  0.38518831820655863
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-08 -0.00586  135.0  146.0           7.0           18.0    0.450450   
2018-01-09 -0.00758  207.0   74.0          41.0           11.0    0.677686   
2018-01-10 -0.01156   94.0  189.0           5.0           22.0    0.308411   
2018-01-11 -0.00952  138.0  143.0           5.0           14.0    0.456693   
2018-01-12 -0.00628  194.0   89.0          33.0           28.0    0.645390   
2018-01-15 -0.00333  263.0   17.0           8.0           80.0    0.163934   
2018-01-16 -0.00345  179.0  104.0          16.0           12.0    0.620155 